In [1]:
from GUI import GUI
from layout.obj.Compo import Compo
import layout.lib.pairing as pairing
import layout.lib.repetition_recognition as rep
import layout.lib.draw as draw
from sklearn.cluster import DBSCAN

import pandas as pd
import math
import numpy as np

In [2]:
gui = GUI(img_file='data/input/2.jpg')

In [3]:
gui.element_detection()
# gui.visualize_element_detection()

*** Please replace the Google OCR key at detect_text/ocr.py line 28 with your own (apply in https://cloud.google.com/vision) ***
[Text Detection Completed in 2.100 s] data/input/2.jpg
(800, 450)
[Compo Detection Completed in 0.737 s] data/input/2.jpg
Merge Complete and Save to data/output\uied\2.jpg Sun Aug  1 11:23:12 2021 



In [4]:
gui.visualize_element_detection()

In [5]:
gui.layout_recognition(check_valid_group=False)
gui.visualize_layout_recognition()

In [6]:
gui.visualize_layout_recognition()

In [4]:
gui.cvt_compos_json_to_dataframe()

In [5]:
gui.compos_df.repetitive_group_recognition()

In [6]:
gui.visualize_compos_df('group')

In [8]:
# gui.compos_df.pair_groups()

In [12]:
# gui.visualize_compos_df('group_pair')

In [4]:
gui.cvt_compos_json_to_dataframe()
gui.compos_df.repetitive_group_recognition()

In [5]:
compos = gui.compos_df
df = compos.compos_dataframe
all_groups = compos.split_groups('group')

In [6]:
g_nt1 = df[df['group'] == 'nt-1']
g_nt2 = df[df['group'] == 'nt-2']
g_t01 = df[df['group'] == 't-0-1']
g_t0 = df[df['group'] == 't-0']

In [7]:
pairing.match_two_groups_by_distance(g_nt1, g_t01)

False

In [9]:
pairing.match_two_groups_by_distance(g_nt2, g_t01)

14 [17]
-174 [-174]
16 [17, 14]
-172 [-174, -174]
16 [17, 14, 16]
-174 [-174, -174, -172]
15 [17, 14, 16, 16]
-174 [-174, -174, -172, -174]
15 [17, 14, 16, 16, 15]
-173 [-174, -174, -172, -174, -174]
16 [17, 14, 16, 16, 15, 15]
-175 [-174, -174, -172, -174, -174, -173]
[17, 14, 16, 16, 15, 15, 16]
[-174, -174, -172, -174, -174, -173, -175]


D:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


True

In [8]:
gui.visualize_compos_df('group')

In [8]:
# gui.layout_recognition(check_valid_group=False)
# gui.visualize_layout_recognition()